# Exploring the different regridded versions of the GCM data

## Load some import constants

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cftime

In [ ]:
# Our two different projections
cp_model_rotated_pole = ccrs.RotatedPole(pole_longitude=177.5, pole_latitude=37.5)
platecarree = ccrs.PlateCarree()

In [ ]:
def plot_with_ts(slices, timestamps, variable='pr'):
    f, axes = plt.subplots(len(timestamps), len(slices), figsize=(20, 20), subplot_kw={'projection': cp_model_rotated_pole})
    f.tight_layout(h_pad=2)
    # make sure axes is 2-d even if only 1 timestamp and or slice
    axes = axes.reshape(len(timestamps),len(slices))
    
    for t, timestamp in enumerate(timestamps):
        for i, data in enumerate(slices):
            ax = axes[t][i]
            ax.coastlines()
            
            x = "longitude"
            y = "latitude"
            transform = platecarree
            if "grid_latitude" in data.coords.keys():
                x = f"grid_longitude"
                y = f"grid_latitude"
                transform = cp_model_rotated_pole
            
            data.sel(time=timestamp)[variable].plot(ax=ax, x=x, y=y, add_colorbar=True, transform = transform, vmin=0)

    plt.show()

## Load the different datasets

In [ ]:
orig_gcm_data = xr.open_mfdataset("../../../../data/60km/rcp85/01/pr/day/*.nc").isel(ensemble_member=0).sel(time=slice("1980-12-01","1982-11-30"))
gcm_data = xr.open_mfdataset("../../../../derived_data/60km/rcp85/01/pr/day/*.nc").isel(ensemble_member=0).sel(time=slice("1980-12-01","1982-11-30"))
cpm_data = xr.open_mfdataset("../../../../data/2.2km/rcp85/01/pr/day/*.nc").isel(ensemble_member=0).sel(time=slice("1980-12-01","1982-11-30"))

regridded_nn_gcm_data = xr.open_mfdataset("../../../../derived_data/60km-nn-regrid-2.2km/rcp85/01/pr/day/*-198*.nc").isel(ensemble_member=0).sel(time=slice("1980-12-01","1982-11-30"))
regridded_lin_gcm_data = xr.open_mfdataset("../../../../derived_data/60km-lin-regrid-2.2km/rcp85/01/pr/day/*-198*.nc").isel(ensemble_member=0).sel(time=slice("1980-12-01","1982-11-30"))

In [ ]:
five_wettest_gcm_days = gcm_data.sum(dim=["projection_x_coordinate", "projection_y_coordinate"]).sortby('pr').time[-5:]
five_wettest_regridded_nn_gcm_days = regridded_nn_gcm_data.sum(dim=["grid_latitude", "grid_longitude"]).sortby('pr').time[-5:]
five_wettest_regridded_lin_gcm_days = regridded_lin_gcm_data.sum(dim=["grid_latitude", "grid_longitude"]).sortby('pr').time[-5:]
five_wettest_cpm_days = cpm_data.sum(dim=["grid_latitude", "grid_longitude"]).sortby('pr').time[-5:]

# Compare wettest days

In [ ]:
five_wettest_gcm_days.values

In [ ]:
five_wettest_regridded_nn_gcm_days.values

In [ ]:
five_wettest_regridded_lin_gcm_days.values

In [ ]:
five_wettest_cpm_days.values

# View some wet days in different versions of data

The raw GCM precip, the GCM precip regrided to match CPM grid according to NN and linear schemes, and the raw CPM precip

## Wettest days according to GCM

In [ ]:
plot_with_ts([orig_gcm_data, gcm_data, regridded_nn_gcm_data, regridded_lin_gcm_data, cpm_data], five_wettest_gcm_days)

## Wettest days according to CPM

In [ ]:
plot_with_ts([gcm_data, regridded_nn_gcm_data, regridded_lin_gcm_data, cpm_data], five_wettest_cpm_days)

Those wettest CPM days don't look that wet on the plot. Maybe the colour scale is different and/or there's something odd going on at the edges